## Google Optimization Toolsで配送計画問題を解いてみる
六本木付近のローソンを車両でまわり、荷物を集荷する想定で、最短経路を解いてみる。車両台数、積載量を制約とし、走行距離が最小となる解を求める。

In [107]:
import folium
import networkx as nx
import osmnx as ox

ox.config(use_cache=True, log_console=True)

# 集荷店舗一覧（六本木付近のローソン)
lawsons = [
           {'nearest_node': 1809615572, 'name': 'アークヒルズ店', 'lat': 35.6681770, 'lon': 139.7397242}, # ここがDepo
           {'nearest_node': 251864110, 'name': '港赤坂九丁目店', 'lat': 35.6680961, 'lon': 139.7328141},
           {'nearest_node': 1760515775, 'name': '城山トラストタワー', 'lat': 35.6649108, 'lon': 139.7431470},
           {'nearest_node': 2162858601, 'name': '合同庁舎第７号館', 'lat': 35.6715977, 'lon': 139.7483260},
           {'nearest_node': 499193143, 'name': '東麻布三丁目', 'lat': 35.6571622, 'lon': 139.7392354},
           {'nearest_node': 1618521241, 'name': '虎ノ門一丁目', 'lat': 35.6685010, 'lon': 139.7489690},
           {'nearest_node': 499189994, 'name': '赤坂六丁目店', 'lat': 35.6700138, 'lon': 139.7337181},
           {'nearest_node': 2207933301, 'name': '100 元麻布店', 'lat': 35.6578560, 'lon': 139.7274620},
           {'nearest_node': 499192852, 'name': '麻布十番一丁目店', 'lat': 35.657044, 'lon': 139.736167},
           {'nearest_node': 1655440289, 'name': '西麻布店', 'lat': 35.6603463, 'lon': 139.7230918},
           {'nearest_node': 4414312668, 'name': '赤坂氷川公園前店', 'lat': 35.6713681, 'lon': 139.7372166},
           {'nearest_node': 1655503992, 'name': '虎ノ門琴平点', 'lat': 35.6703080, 'lon': 139.7487411},
           {'nearest_node': 1482491357, 'name': '新橋六丁目店', 'lat': 35.661263, 'lon': 139.754532}]

# 六本木交差点から半径1000ｍの道路ネットワークをOSMから生成する
ROPPONGI = (35.663236, 139.732275)
G = ox.graph_from_point(ROPPONGI, distance=3000, network_type='drive', simplify=False)
map = folium.Map(location=ROPPONGI, tiles='cartodbpositron', zoom_start=15)

for i in lawsons:
    folium.Marker([i['lat'], i['lon']], popup=i['name']).add_to(map)

# ひとまず地図上にプロット
map

In [108]:
from ortools.constraint_solver import pywrapcp

# 車両の色
vehicle_color = ['red', 'blue','green','orange','yellow']

# 積載制約上のゆとり
slack_max = 0

# 拠点ごとの集荷需要
demands = [1, 19, 21, 6, 19, 7, 12, 16, 6, 16, 8, 14, 21]

# 六本木交差点から半径1000ｍの道路ネットワークをOSMから生成する
ROPPONGI = (35.663236, 139.732275)
G = ox.graph_from_point(ROPPONGI, distance=3000, network_type='drive', simplify=False)
map = folium.Map(location=ROPPONGI, tiles='cartodbpositron', zoom_start=7)


# 結果コンテナ
result = [[], [], []]

def distance(x, y):
    """
    ノード間の距離をグラフから返す. コストは距離固定.
    :param x: (Int) 始点node id
    :param y: (Int) 終点node id
    :return: Int: 距離(m)
    """
    # print('x:{}, y:{}'.format(x,y))
    return nx.shortest_path_length(G, x, y, weight='length')


class CreateDistanceCallback(object):
    """二点間の距離を返すCallbackを定義"""

    def __init__(self, locations):
        """コストマトリックスを生成"""
        size = len(locations)
        self.matrix = {}

        for from_node in range(size):
            self.matrix[from_node] = {}
            for to_node in range(size):
                x = locations[from_node]
                y = locations[to_node]
                self.matrix[from_node][to_node] = distance(x, y)


    def memoize(f):
        """キャッシュ用デコレータ"""
        cache = {}

        def helper(*args):
            if args not in cache:
                cache[args] = f(*args)
            return cache[args]

        return helper


    @memoize
    def Distance(self, from_node, to_node):
        """
        ノード間の距離を生成したマトリックスから返す
        ソルバーはIntしか取らないので、Intに変換してから返す
        """
        return int(self.matrix[from_node][to_node])


# 集荷需要コールバック関数
class CreateDemandCallback(object):
    """各拠点での集荷需要を返すCallbackクラスを定義"""

    def __init__(self, demands):
        self.matrix = demands

    def Demand(self, from_node, to_node):
        return self.matrix[from_node]
    
# 各拠点とそれぞれの集荷需要データを生成
def create_data_array():
    locations = []
    for c, i in enumerate(lawsons):
        node_point = (i['lat'], i['lon'])
        node = ox.get_nearest_node(G, node_point)
        lawsons[c]['nearest_node'] = node
        locations.append(node)

    data = [locations, demands]
    return data


In [109]:
def main(vehicle_capacity=56, num_vehicles=3):
    
    # 拠点と集荷需要のlistを生成
    data = create_data_array()
    locations = data[0]
    demands = data[1]
    num_locations = len(locations)
    
     # 最初と最後をデポに指定
    depot = 0 
    
    # ルーティングモデルインスタンス生成
    routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

    # 解を探す時間制限(ms)
    search_parameters.time_limit_ms = 30000

    # node間の距離を返すコールバック関数を渡す
    dist_between_locations = CreateDistanceCallback(locations)
    dist_callback = dist_between_locations.Distance

    # ソルバーにコールバックを登録
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)

    # 拠点ごとの集荷需要を返すコールバック関数を渡す
    demands_at_locations = CreateDemandCallback(demands)
    demands_callback = demands_at_locations.Demand

    # スタート地点での積載量をゼロに固定する
    fix_start_cumul_to_zero = True
    demand = "Demand"

    # 積載需要をモデルに追加する
    routing.AddDimension(demands_callback, slack_max, vehicle_capacity, fix_start_cumul_to_zero, demand)

    # 解を求め、解があれば返す
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
            # 結果
            print("全ルート合計距離: {}m".format(str(assignment.ObjectiveValue())))
            
            #　車両
            for vehicle_nbr in range(num_vehicles):
                # 結果を格納する変数を初期化
                index = routing.Start(vehicle_nbr)
                index_next = assignment.Value(routing.NextVar(index))
                route = ''
                route_dist = 0
                route_demand = 0
                depo_node = routing.IndexToNode(index)
                
                # 始点終点を黒アイコンで地図にプロット
                folium.Marker(
                    [lawsons[depo_node]['lat'], lawsons[depo_node]['lon']],
                    popup=folium.Popup("{}({})".format(lawsons[depo_node]['name'], depo_node), parse_html=True),
                    icon=folium.Icon(color='black')
                ).add_to(map)

                #　ルート
                while not routing.IsEnd(index_next):
                    node_index = routing.IndexToNode(index)
                    node_index_next = routing.IndexToNode(index_next)
                    route += lawsons[node_index]['name'] + " -> "
                    # 距離を加算
                    route_dist += dist_callback(node_index, node_index_next)
                    # 需要を加算
                    route_demand += demands[node_index_next]
                    index = index_next
                    index_next = assignment.Value(routing.NextVar(index))
                    start_node = lawsons[node_index]['nearest_node']
                    end_node = lawsons[node_index_next]['nearest_node']

                    # ノードアイコンとノード間のルート線を地図にプロット
                    folium_route = nx.shortest_path(G, start_node, end_node, weight='length')
                    ox.plot_route_folium(G, folium_route, route_map=map, route_color=vehicle_color[vehicle_nbr], route_width=8, route_opacity=0.8)
                    folium.Marker(
                        [lawsons[node_index_next]['lat'], lawsons[node_index_next]['lon']],
                        popup=folium.Popup("{}({})".format(lawsons[node_index_next]['name'], node_index_next), parse_html=True),
                        icon=folium.Icon(color=vehicle_color[vehicle_nbr])
                    ).add_to(map)

                # 最後のノードとデポまでの帰路をプロット
                folium_route = nx.shortest_path(G, end_node, lawsons[depo_node]['nearest_node'], weight='length')
                ox.plot_route_folium(G, folium_route, route_map=map, route_color=vehicle_color[vehicle_nbr], route_opacity=0.5)

                node_index = routing.IndexToNode(index)
                node_index_next = routing.IndexToNode(index_next)
                route += lawsons[node_index]['name'] + " -> " + lawsons[node_index_next]['name']
                route_dist += dist_callback(node_index, node_index_next)
                print("  ======")
                print("  {}号車({})".format(str(vehicle_nbr+1), vehicle_color[vehicle_nbr]))
                print("　　　ルート: {}".format(route))
                print("　　　走行距離:{}m".format(str(route_dist)))
                print("　　　積載量:{}".format(str(route_demand)))
    else:
            print('解がありません')



## 解を求める

### ケース１：　車両の最大積載量 56, 車両台数3台

In [110]:
map = folium.Map(location=ROPPONGI, tiles='cartodbpositron', zoom_start=7)
main(vehicle_capacity=56, num_vehicles=3)

全ルート合計距離: 16758m
  1号車(red)
　　　ルート: アークヒルズ店 -> 虎ノ門琴平点 -> 100 元麻布店 -> 麻布十番一丁目店 -> 東麻布三丁目 -> アークヒルズ店
　　　走行距離:6597m
　　　積載量:55
  2号車(blue)
　　　ルート: アークヒルズ店 -> 合同庁舎第７号館 -> 虎ノ門一丁目 -> 新橋六丁目店 -> 城山トラストタワー -> アークヒルズ店
　　　走行距離:5256m
　　　積載量:55
  3号車(green)
　　　ルート: アークヒルズ店 -> 赤坂氷川公園前店 -> 赤坂六丁目店 -> 港赤坂九丁目店 -> 西麻布店 -> アークヒルズ店
　　　走行距離:4905m
　　　積載量:55


In [111]:
map

### ケース２：　車両の最大積載量 45, 車両台数4台

In [112]:
map = folium.Map(location=ROPPONGI, tiles='cartodbpositron', zoom_start=7)
main(vehicle_capacity=45, num_vehicles=4)

全ルート合計距離: 17962m
  1号車(red)
　　　ルート: アークヒルズ店 -> 100 元麻布店 -> 麻布十番一丁目店 -> 東麻布三丁目 -> アークヒルズ店
　　　走行距離:4854m
　　　積載量:41
  2号車(blue)
　　　ルート: アークヒルズ店 -> 虎ノ門一丁目 -> 新橋六丁目店 -> 西麻布店 -> アークヒルズ店
　　　走行距離:7788m
　　　積載量:44
  3号車(green)
　　　ルート: アークヒルズ店 -> 合同庁舎第７号館 -> 虎ノ門琴平点 -> 城山トラストタワー -> アークヒルズ店
　　　走行距離:3357m
　　　積載量:41
  4号車(orange)
　　　ルート: アークヒルズ店 -> 赤坂氷川公園前店 -> 赤坂六丁目店 -> 港赤坂九丁目店 -> アークヒルズ店
　　　走行距離:1963m
　　　積載量:39


In [113]:
map

### ケース3：　車両の最大積載量 38 車両台数5台

In [114]:
map = folium.Map(location=ROPPONGI, tiles='cartodbpositron', zoom_start=7)
main(vehicle_capacity=38, num_vehicles=5)

全ルート合計距離: 21771m
  1号車(red)
　　　ルート: アークヒルズ店 -> 港赤坂九丁目店 -> 西麻布店 -> アークヒルズ店
　　　走行距離:4556m
　　　積載量:35
  2号車(blue)
　　　ルート: アークヒルズ店 -> 100 元麻布店 -> 東麻布三丁目 -> アークヒルズ店
　　　走行距離:4790m
　　　積載量:35
  3号車(green)
　　　ルート: アークヒルズ店 -> 虎ノ門琴平点 -> 赤坂氷川公園前店 -> 赤坂六丁目店 -> アークヒルズ店
　　　走行距離:3543m
　　　積載量:34
  4号車(orange)
　　　ルート: アークヒルズ店 -> 城山トラストタワー -> 麻布十番一丁目店 -> アークヒルズ店
　　　走行距離:3682m
　　　積載量:27
  5号車(yellow)
　　　ルート: アークヒルズ店 -> 合同庁舎第７号館 -> 虎ノ門一丁目 -> 新橋六丁目店 -> アークヒルズ店
　　　走行距離:5200m
　　　積載量:34


In [115]:
map

### 結果、ケース1が一番よさげ